# Introduction to SageMaker JumpStart - text generation with Mistral models

---
In this demo notebook, we demonstrate how to use the SageMaker Python SDK to deploy Mistral models for text generation tasks.

---

## Setup
***

In [2]:
model_id = "huggingface-llm-mistral-7b-instruct"

In [9]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id)
predictor = model.deploy()

-----------!

### Supported parameters

***
This model supports many parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments
***

## Instruction prompts
***
The examples in this section demonstrate queries to the Mistral 7B Instruct model. This involves special token formatting within the prompt input. The base, pre-trained Mistral model is not fine-tuned to perform this instruction task -- please use the prompts in the next section instead.
***

In [4]:
from typing import Dict, List


def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for user, answer in zip(instructions[::2], instructions[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])
    prompt.extend(["<s>", "[INST] ", (instructions[-1]["content"]).strip(), " [/INST] "])
    return "".join(prompt)


def print_prompt_and_response(prompt: str, response: str) -> None:
    bold, unbold = '\033[1m', '\033[0m'
    print(f"{bold}> Input{unbold}\n{prompt}\n\n{bold}> Output{unbold}\n{response[0]['generated_text']}\n")

In [5]:
instructions = [{"role": "user", "content": "what is the recipe of mayonnaise?"}]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True}
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
<s>[INST] what is the recipe of mayonnaise? [/INST] 

> Output
 here is a simple recipe for homemade mayonnaise using a blender or food processor:

Ingredients:
- 1 cup (240 ml) vegetable oil (canola, safflower, or sunflower oil work best)
- 1 large egg yolk
- 1 tablespoon (15 ml) white wine vinegar or apple cider vinegar
- 1 tablespoon (15 ml) lemon juice
- 1 teaspoon (5 g) Dijon mustard
- 1 teaspoon (5 g) salt
- 1/2 teaspoon (2.5 g) sugar (optional)
- 1-2 tablespoons (15-30 ml) cold water (if needed)

Instructions:

1. In a blender or food processor, add the egg yolk, vinegar, lemon juice, mustard, salt, and sugar (if using). Pulse a few times to combine.
2. With the machine running, slowly drizzle in about 1/4 cup (60 ml) of oil. duit the oil in a thin, steady stream until all the oil is incorporated



In [6]:
instructions = [
    {"role": "user", "content": "I am going to Paris, what should I see?"},
    {
        "role": "assistant",
        "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
    },
    {"role": "user", "content": "What is so great about #1?"},
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True}
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
<s>[INST] I am going to Paris, what should I see? [/INST] Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.</s><s>[INST] What is so great about #1? [/INST] 

> Output
1. The Eiffel Tower 

In [7]:
instructions = [
    {
        "role": "user",
        "content": "In Bash, how do I list all text files in the current directory (excluding subdirectories) that have been modified in the last month?",
    }
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.2}
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
<s>[INST] In Bash, how do I list all text files in the current directory (excluding subdirectories) that have been modified in the last month? [/INST] 

> Output
You can use the `find` command in combination with `-type`, `-name`, `-mtime`, and `-print` options to list all text files (with `.txt` extension) in the current directory that have been modified within the last month. Here's the command:

```bash
find . -type f -name "*.txt" -mtime -30 -print
```

Let's break down the command:

- `find .`: Start the search from the current directory.
- `-type f`: Look for files only.
- `-name "*.txt"`: Look for files with the `.txt` extension.
- `-mtime -30`: Look for files modified within the last 30 days.
- `-print`: Print the names of the files that match the criteria.

This command will list all text files (with the `.txt` extension) in the current directory that have been modified within the last month.



In [10]:
instructions = [
    {
        "role": "user",
        "content": "What is the difference between inorder and preorder traversal? Give an example in Python.",
    }
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.2}
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
<s>[INST] What is the difference between inorder and preorder traversal? Give an example in Python. [/INST] 

> Output
Inorder and preorder are two common tree traversal methods used to visit all the nodes in a binary tree. Here's a brief explanation of both traversals and the differences between them:

**Inorder Traversal:** In this traversal method, the left subtree is visited first, then the root node, and finally the right subtree. This results in a traversal where the nodes are visited in their respective in-order or traversal sequence, which is the left subtree, root, and right subtree.

**Preorder Traversal:** In preorder traversal, the root node is visited first, followed by the left subtree, and finally the right subtree. This results in a traversal where the nodes are visited in the following order: root, left subtree, and right subtree.

Here's an example of both inorder and preorder traversals in Python:

```python
class Node:
    def __init__(self, key):
        se

## Pre-trained model prompts
***
The examples in this section demonstrate how to perform text generation on the base, pre-trained Mistral model. If you have deployed the instruction-tuned model, please use prompt formatting in the previous section instead.
***

In [11]:
prompt = "Write a program to compute factorial in python:"
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 200,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Write a program to compute factorial in python:

> Output


Here is a simple Python program to compute the factorial of a given number using a recursive function:

```python
# Function to compute factorial using recursion
def factorial(n):
    # Base case: factorial of 0 is 1
    if n == 0:
        return 1
    # Recursive case: n! = n * (n-1)!
    else:
        return n * factorial(n-1)

# Test the function
num = int(input("Enter a number: "))
result = factorial(num)
print(f"The factorial of {num} is {result}")
```

This program defines a function `factorial` that takes an integer `n` as an argument and returns its factorial. The base case for the recursion is when `n` is equal to 0, in which



In [16]:
prompt = "Building a website can be done in 10 simple steps:"
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 110,
        "no_repeat_ngram_size": 3,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Building a website can be done in 10 simple steps:

> Output


1. Define your website's purpose and target audience.
2. Choose a domain name and register it.
3. Select a website builder or content management system (CMS) and sign up for a hosting plan.
4. Plan your website's structure and create a sitemap.
5. Design your website's layout and create wireframes and mockups.
6. Write and optimize your website's content.
7. Add images, videos, and other media to your website.




In [13]:
prompt = """\
Translate English to French:

sea otter => loutre de mer

peppermint => menthe poivrée

plush girafe => girafe peluche

cheese =>"""
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 3,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Translate English to French:

sea otter => loutre de mer

peppermint => menthe poivrée

plush girafe => girafe peluche

cheese =>

> Output
 fromage




In [14]:
prompt = """\
Tweet: "I hate it when my phone battery dies."
Sentiment: Negative
###
Tweet: "My day has been :+1:"
Sentiment: Positive
###
Tweet: "This is the link to the article"
Sentiment: Neutral
###
Tweet: "This new music video was incredibile"
Sentiment:"""
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 2,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Tweet: "I hate it when my phone battery dies."
Sentiment: Negative
###
Tweet: "My day has been :+1:"
Sentiment: Positive
###
Tweet: "This is the link to the article"
Sentiment: Neutral
###
Tweet: "This new music video was incredibile"
Sentiment:

> Output
 Positive



In [15]:
prompt = "Could you remind me when was the C programming language invented?"
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 50,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Could you remind me when was the C programming language invented?

> Output


Yes, I can help you with that! The C programming language was developed by Dennis Ritchie at Bell Labs in 1972. It was initially used to write the UNIX operating system and its utilities. The



In [17]:
prompt = "What is the recipe for a delicious lemon cheesecake?"
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 200,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
What is the recipe for a delicious lemon cheesecake?

> Output
 Here's a simple and delicious recipe for a classic lemon cheesecake that is sure to impress your guests.

Ingredients:

For the crust:
- 1 1/2 cups graham cracker crumbs
- 1/4 cup granulated sugar
- 6 tablespoons unsalted butter, melted

For the filling:
- 24 ounces cream cheese, softened
- 1 1/3 cups granulated sugar
- 4 large eggs
- 1 cup sour cream
- 1/4 cup all-purpose flour
- 1 tablespoon lemon zest
- 1/2 cup freshly squeezed lemon juice

For the topping:
- 1 cup sour cream
- 1/3 cup granulated sugar
- 1 teaspoon vanilla extract

Instructions:

1. Preheat the



In [24]:
prompt = "Compose a Legal Auto Claims Policy from Insurer Chubb. The Legal Policy must have different subsections. The subsections are \
a) Automobile Liability Insurance. b) Automobile Medical Payments. c) Automobile Debt Indemnity Insurance. \
d) Uninsured Motorists Insurance. e) Personal Injury Protection. f) Collision Insurance. g) Comprehensive Insurance. h) Rental Reimbursement Insurance. i) Towing and Labor Coverage.\
j) Waiver of Deductible. Each subsection must include Exclusions.  Each subsection must include LIMITS OF LIABILITY; What is not covered. "
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 1200,
        "no_repeat_ngram_size": 3,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Compose a Legal Auto Claims Policy from Insurer Chubb. The Legal Policy must have different subsections. The subsections are a) Automobile Liability Insurance. b) Automobile Medical Payments. c) Automobile Debt Indemnity Insurance. d) Uninsured Motorists Insurance. e) Personal Injury Protection. f) Collision Insurance. g) Comprehensive Insurance. h) Rental Reimbursement Insurance. i) Towing and Labor Coverage.j) Waiver of Deductible. Each subsection must include Exclusions.  Each subsection must include LIMITS OF LIABILITY; What is not covered. 

> Output


I. AUTOMOBILE LIABILITY INSURANCE

a. Description: This coverage provides protection against legal liability for bodily injury or property damage caused by an insured automobile.

b. Limits of Liability: The policy will provide coverage up to a limit of $500,000 for each accident.

c. Exclusions:
1. Coverage does not apply to damages caused intentionally by an insured.
2. Coverage does not apply to damages caused by a vehicl

In [26]:
#!pip install Faker

In [101]:
from faker import Faker
fake = Faker()

primary_name = fake.name()
primary_address = fake.address()
primary_city = fake.city()
primary_state = fake.state()
primary_zip = fake.zipcode()
primary_email_address   = fake.email()
secondary_name = fake.name()
secondary_address = fake.address()
secondary_email_address   = fake.email()

In [102]:
print(primary_name,primary_address,email_address)

Kim Young 59881 Haley River Apt. 781
Adammouth, RI 96334 pknapp@example.com


In [95]:
prompt = "Compose a Legal Auto Claims Policy between the Insurer Chubb and {} \
\nAddress: {} \nEmail:   {}   \
\n The Legal Auto Claims Policy must include the Address and Email Information at the beginning of the Legal Auto Claims Policy.  \
\nThe Legal Policy must have different subsections.\
The subsections are \
a) Automobile Liability Insurance. b) Automobile Medical Payments. c) Automobile Debt Indemnity Insurance. \
d) Uninsured Motorists Insurance. e) Default Provisions. f) Personal Injury Protection. g) Collision Insurance. h) Comprehensive Insurance. i) Rental Reimbursement Insurance. j) Towing and Labor Coverage.\
k) Waiver of Deductible. Each subsection must include Exclusions.  Each subsection must include LIMITS OF LIABILITY; What is not covered. \
The Legal Auto Claims Policy must also include the following mailing contact information {}, {},{} \
".format(primary_name,primary_address,email_address,secondary_name,secondary_address,secondary_email_address)

In [96]:
print(prompt)

Compose a Legal Auto Claims Policy between the Insurer Chubb and John Rivas 
Address: 9367 Saunders Lane
Coreyland, WY 27862 
Email:   pknapp@example.com   
 The Legal Auto Claims Policy must include the Address and Email Information at the beginning of the Legal Auto Claims Policy.  
The Legal Policy must have different subsections.The subsections are a) Automobile Liability Insurance. b) Automobile Medical Payments. c) Automobile Debt Indemnity Insurance. d) Uninsured Motorists Insurance. e) Default Provisions. f) Personal Injury Protection. g) Collision Insurance. h) Comprehensive Insurance. i) Rental Reimbursement Insurance. j) Towing and Labor Coverage.k) Waiver of Deductible. Each subsection must include Exclusions.  Each subsection must include LIMITS OF LIABILITY; What is not covered. The Legal Auto Claims Policy must also include the following mailing contact information Heidi Hall, USNV Cruz
FPO AE 98217,hernandezkathryn@example.com 


In [86]:
prompt

'Compose a Legal Auto Claims Policy between the Insurer Chubb and Vicki Brown \nAddress: 4047 Catherine Islands\nPort Kentton, CA 13429 \nEmail:   pknapp@example.com   \n The Legal Auto Claims Policy must include the Address and Email Information.  \nThe Legal Policy must have different subsections.The subsections are a) Automobile Liability Insurance. b) Automobile Medical Payments. c) Automobile Debt Indemnity Insurance. d) Uninsured Motorists Insurance. e) Personal Injury Protection. f) Collision Insurance. g) Comprehensive Insurance. h) Rental Reimbursement Insurance. i) Towing and Labor Coverage.j) Waiver of Deductible. Each subsection must include Exclusions.  Each subsection must include LIMITS OF LIABILITY; What is not covered. The Legal Auto Claims Policy must also include the following mailing contact information Jason Weeks, 4452 Martinez Branch\nPort Michael, MT 89571,vbradford@example.com '

In [97]:
prompt = "Compose a Legal Auto Claims Policy between the Insurer Chubb and {} \
\nAddress: {} \nEmail:   {}   \
\n The Legal Auto Claims Policy must include the Address and Email Information at the beginning of the Legal Auto Claims Policy.  \
\nThe Legal Policy must have different subsections.\
The subsections are \
a) Automobile Liability Insurance. b) Automobile Medical Payments. c) Automobile Debt Indemnity Insurance. \
d) Uninsured Motorists Insurance. e) Default Provisions. f) Personal Injury Protection. g) Collision Insurance. h) Comprehensive Insurance. i) Rental Reimbursement Insurance. j) Towing and Labor Coverage.\
k) Waiver of Deductible. Each subsection must include Exclusions.  Each subsection must include LIMITS OF LIABILITY; What is not covered. \
The Legal Auto Claims Policy must also include the following mailing contact information {}, {},{} \
".format(primary_name,primary_address,email_address,secondary_name,secondary_address,secondary_email_address)

payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 1200,
        "no_repeat_ngram_size": 3,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Compose a Legal Auto Claims Policy between the Insurer Chubb and John Rivas 
Address: 9367 Saunders Lane
Coreyland, WY 27862 
Email:   pknapp@example.com   
 The Legal Auto Claims Policy must include the Address and Email Information at the beginning of the Legal Auto Claims Policy.  
The Legal Policy must have different subsections.The subsections are a) Automobile Liability Insurance. b) Automobile Medical Payments. c) Automobile Debt Indemnity Insurance. d) Uninsured Motorists Insurance. e) Default Provisions. f) Personal Injury Protection. g) Collision Insurance. h) Comprehensive Insurance. i) Rental Reimbursement Insurance. j) Towing and Labor Coverage.k) Waiver of Deductible. Each subsection must include Exclusions.  Each subsection must include LIMITS OF LIABILITY; What is not covered. The Legal Auto Claims Policy must also include the following mailing contact information Heidi Hall, USNV Cruz
FPO AE 98217,hernandezkathryn@example.com 

> Output


Legal Auto Claims Poli

## Clean up the endpoint

In [18]:
#predictor.delete_model()
#predictor.delete_endpoint()